<!--<badge>--><a href="https://colab.research.google.com/github/gradsflow/gradsflow/blob/main/examples/nbs/06-AutoModel_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

This example shows how to search hyperparameters for a model with Tuner module.

In [ ]:
import os

# you can remove this
os.chdir("../../")
os.environ["LOGURU_LEVEL"] = "INFO"

In [ ]:
from gradsflow import Model
from gradsflow.tuner.tuner import Tuner

In [ ]:
from timm import create_model
from ray import tune
from gradsflow.data.image import get_fake_data

from gradsflow import AutoDataset

In [ ]:
image_size = (64, 64)
fake_data = get_fake_data(image_size, num_workers=0)
train_ds, train_dl = fake_data.dataset, fake_data.dataloader

fake_data = get_fake_data(image_size, num_workers=0)
val_ds, val_dl = fake_data.dataset, fake_data.dataloader

num_classes = train_ds.num_classes
autodataset = AutoDataset(train_dl, val_dl, num_classes=num_classes)

In [ ]:
from gradsflow.tuner.tuner import Tuner
from gradsflow.tuner.automodel import AutoModelV2

## Registering hyperparameters

Gradsflow AutoModel provides you two main ways to register your hyperparameters.

Easiest way is to compile the model and the values will be registered automatically.
In this example we will hyperparameter search for ConvNet architecture, optimizer and learning rate.

In [ ]:
tuner = Tuner()


cnn1 = create_model("resnet18", pretrained=False, num_classes=num_classes)
cnn2 = create_model("efficientnet_b0", pretrained=False, num_classes=num_classes)
cnns = tuner.suggest_complex("learner", cnn1, cnn2)

2021-10-08 23:10:26,751	INFO services.py:1250 -- View the Ray dashboard at http://127.0.0.1:8265


In [ ]:
model = AutoModelV2(cnns)

model.compile(
    loss="crossentropyloss",
    optimizer=tune.choice(("adam", "sgd")),
    learning_rate=tune.loguniform(1e-5, 1e-3),
    metrics="accuracy",
)

In [ ]:
model.hp_tune(tuner, autodataset, epochs=1)

2021-10-08 23:10:28,168	WARNING function_runner.py:558 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,learner,learning_rate,optimizer
trainable_d37b0_00000,PENDING,,0,1.57275e-05,sgd


Trial name,status,loc,learner,learning_rate,optimizer
trainable_d37b0_00000,RUNNING,,0,1.57275e-05,sgd


Trial name,status,loc,learner,learning_rate,optimizer
trainable_d37b0_00000,RUNNING,,0,1.57275e-05,sgd


Trial name,status,loc,learner,learning_rate,optimizer
trainable_d37b0_00000,RUNNING,,0,1.57275e-05,sgd


Result for trainable_d37b0_00000:
  date: 2021-10-08_23-10-46
  done: false
  experiment_id: ca54b6cd04ef4c1388cbd87f8b775b8c
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 1
  node_ip: 192.168.38.84
  pid: 3871
  should_checkpoint: true
  time_since_restore: 13.018282890319824
  time_this_iter_s: 13.018282890319824
  time_total_s: 13.018282890319824
  timestamp: 1633714846
  timesteps_since_restore: 0
  train_accuracy: tensor(0.1200)
  train_loss: 2.309937852597979
  training_iteration: 1
  trial_id: d37b0_00000
  val_accuracy: tensor(0.1000)
  val_loss: 2.297529087043675
  
Result for trainable_d37b0_00000:
  date: 2021-10-08_23-10-46
  done: true
  experiment_id: ca54b6cd04ef4c1388cbd87f8b775b8c
  experiment_tag: 0_learner=0,learning_rate=1.5728e-05,optimizer=sgd
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 1
  node_ip: 192.168.38.84
  pid: 3871
  should_checkpoint: true
  time_since_restore: 13.018282890319824
  time_this_iter_s: 13.018

Trial name,status,loc,learner,learning_rate,optimizer,iter,total time (s),val_loss,train_loss
trainable_d37b0_00000,TERMINATED,,0,1.57275e-05,sgd,1,13.0183,2.29753,2.30994


2021-10-08 23:10:46,283	INFO tune.py:617 -- Total run time: 18.12 seconds (17.96 seconds for the tuning loop).
2021-10-08 23:10:46.296 | INFO     | gradsflow.tuner.automodel:hp_tune:148 - 🎉 Best HyperParameters found: {'learner': 0, 'optimizer': 'sgd', 'learning_rate': 1.5727544813565454e-05}


### The second way to register hyperparameters is to use Tuner module.

In [ ]:
tuner = Tuner()


cnn1 = create_model("resnet18", pretrained=False, num_classes=num_classes)
cnn2 = create_model("efficientnet_b0", pretrained=False, num_classes=num_classes)
cnns = tuner.suggest_complex("learner", cnn1, cnn2)

In [ ]:
tuner.choice("optimizer", "adam", "sgd")
tuner.loguniform("learning_rate", 1e-5, 1e-3)
tuner.scalar("loss", "crossentropyloss")

In [ ]:
model = AutoModelV2(cnns)
model.hp_tune(tuner, autodataset, epochs=1)

Trial name,status,loc,learner,learning_rate,optimizer
trainable_de902_00000,PENDING,,0,0.000854305,sgd


Trial name,status,loc,learner,learning_rate,optimizer
trainable_de902_00000,RUNNING,,0,0.000854305,sgd


Trial name,status,loc,learner,learning_rate,optimizer
trainable_de902_00000,RUNNING,,0,0.000854305,sgd


Trial name,status,loc,learner,learning_rate,optimizer
trainable_de902_00000,RUNNING,,0,0.000854305,sgd


Result for trainable_de902_00000:
  date: 2021-10-08_23-11-04
  done: false
  experiment_id: 912d8ca54ef344f1bc97f90d7ac37b9a
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 1
  node_ip: 192.168.38.84
  pid: 3885
  should_checkpoint: true
  time_since_restore: 11.700548887252808
  time_this_iter_s: 11.700548887252808
  time_total_s: 11.700548887252808
  timestamp: 1633714864
  timesteps_since_restore: 0
  train_loss: 2.3229162955051827
  training_iteration: 1
  trial_id: de902_00000
  val_loss: 2.3163452613122093
  
Result for trainable_de902_00000:
  date: 2021-10-08_23-11-04
  done: true
  experiment_id: 912d8ca54ef344f1bc97f90d7ac37b9a
  experiment_tag: 0_learner=0,learning_rate=0.00085431,optimizer=sgd
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 1
  node_ip: 192.168.38.84
  pid: 3885
  should_checkpoint: true
  time_since_restore: 11.700548887252808
  time_this_iter_s: 11.700548887252808
  time_total_s: 11.700548887252808
  timestamp: 1

Trial name,status,loc,learner,learning_rate,optimizer,iter,total time (s),val_loss,train_loss
trainable_de902_00000,TERMINATED,,0,0.000854305,sgd,1,11.7005,2.31635,2.32292


2021-10-08 23:11:04,392	INFO tune.py:617 -- Total run time: 17.63 seconds (17.51 seconds for the tuning loop).
2021-10-08 23:11:04.400 | INFO     | gradsflow.tuner.automodel:hp_tune:148 - 🎉 Best HyperParameters found: {'learner': 0, 'optimizer': 'sgd', 'learning_rate': 0.0008543052642522848, 'loss': 'crossentropyloss'}
